In [61]:
import pandas as pd
import os

os.chdir('/Users/kanna/Sandbox/kaggle/allstate_purchase/')
train = pd.read_csv('train.csv')
train_final = train[train['record_type']==1]
train_final = train_final.fillna(train_final.median())
test_final = pd.read_csv('test_final.csv')
test_final = test_final.fillna(test_final.median())
#X_train = train_final[['day', 'state', 'location', 'group_size', 'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest', 'age_youngest', 'married_couple', 'duration_previous']]
#X_train = train_final[['day', 'location', 'group_size', 'homeowner', 'car_age', 'risk_factor', 'age_oldest', 'age_youngest', 'married_couple', 'duration_previous']]
#X_train = X_train.fillna(X_train.median())
A_train = train_final['A']
B_train = train_final['B']
C_train = train_final['C']
D_train = train_final['D']
E_train = train_final['E']
F_train = train_final['F']
G_train = train_final['G']
#X_test = test_final[['day', 'state', 'location', 'group_size', 'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest', 'age_youngest', 'married_couple', 'duration_previous']]
#X_test = test_final[['day', 'location', 'group_size', 'homeowner', 'car_age', 'risk_factor', 'age_oldest', 'age_youngest', 'married_couple', 'duration_previous']]
#X_test = X_test.fillna(X_test.median())
X_train.head()

,day,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,duration_previous
8,0,IN,10001,2,0,2,g,3,46,42,1,2
14,3,NY,10006,1,0,10,e,4,28,28,0,13
22,4,PA,10008,1,0,11,c,3,43,43,0,4
26,4,WV,10014,2,1,3,d,3,62,60,1,3
32,1,MO,10015,1,0,5,d,3,32,28,1,2


In [77]:
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [68]:
# dummify categorical variables
dummy_day = pd.get_dummies(train_final['day'], prefix='day')
print dummy_day.head(1)
dummy_state = pd.get_dummies(train_final['state'], prefix='state')
print dummy_state.head(1)
dummy_group_size = pd.get_dummies(train_final['group_size'], prefix='group_size')
print dummy_group_size.head(1)
dummy_car_value = pd.get_dummies(train_final['car_value'], prefix='car_value')
print dummy_car_value.head(1)
dummy_risk_factor = pd.get_dummies(train_final['risk_factor'], prefix='risk_factor')
print dummy_risk_factor.head(1)

   day_0  day_1  day_2  day_3  day_4  day_5  day_6
8      1      0      0      0      0      0      0

[1 rows x 7 columns]
   state_AL  state_AR  state_CO  state_CT  state_DC  state_DE  state_FL  \
8         0         0         0         0         0         0         0   

   state_GA  state_IA  state_ID  state_IN  state_KS  state_KY  state_MD  \
8         0         0         0         1         0         0         0   

   state_ME  state_MO  state_MS  state_MT  state_ND  state_NE      
8         0         0         0         0         0         0 ...  

[1 rows x 36 columns]
   group_size_1  group_size_2  group_size_3  group_size_4
8             0             1             0             0

[1 rows x 4 columns]
   car_value_a  car_value_b  car_value_c  car_value_d  car_value_e  \
8            0            0            0            0            0   

   car_value_f  car_value_g  car_value_h  car_value_i  
8            0            1            0            0  

[1 rows x 9 columns]
  

In [76]:
# create a clean data frame for the regression
cols_to_keep = ['day', 'homeowner', 'car_age', 'age_oldest', 'age_youngest', 'married_couple', 'duration_previous']
X_train = train_final[cols_to_keep].join(dummy_day.ix[:, 'day_1':]).join(dummy_state.ix[:, 'state_AR':]).join(dummy_group_size.ix[:, 'group_size_2':]).join(dummy_car_value.ix[:, 'car_value_b':]).join(dummy_risk_factor.ix[:, 'risk_factor_1.0':])

# manually add the intercept
X_train['intercept'] = 1.0
X_train.head()

,day,homeowner,car_age,age_oldest,age_youngest,married_couple,duration_previous,day_1,day_2,day_3,day_4,day_5,day_6,state_AR,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,
8,0,0,2,46,42,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,...
14,3,0,10,28,28,0,13,0,0,1,0,0,0,0,0,0,0,0,0,0,...
22,4,0,11,43,43,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,...
26,4,1,3,62,60,1,3,0,0,0,1,0,0,0,0,0,0,0,0,0,...
32,1,0,5,32,28,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,...


In [82]:
# dummify categorical variables for test set
dummy_day = pd.get_dummies(test_final['day'], prefix='day')
#print dummy_day.head(1)
dummy_state = pd.get_dummies(test_final['state'], prefix='state')
#print dummy_state.head(1)
dummy_group_size = pd.get_dummies(test_final['group_size'], prefix='group_size')
#print dummy_group_size.head(1)
dummy_car_value = pd.get_dummies(test_final['car_value'], prefix='car_value')
#print dummy_car_value.head(1)
dummy_risk_factor = pd.get_dummies(test_final['risk_factor'], prefix='risk_factor')
#print dummy_risk_factor.head(1)

cols_to_keep = ['day', 'homeowner', 'car_age', 'age_oldest', 'age_youngest', 'married_couple', 'duration_previous']
X_test = test_final[cols_to_keep].join(dummy_day.ix[:, 'day_1':]).join(dummy_state.ix[:, 'state_AR':]).join(dummy_group_size.ix[:, 'group_size_2':]).join(dummy_car_value.ix[:, 'car_value_b':]).join(dummy_risk_factor.ix[:, 'risk_factor_1.0':])

# manually add the intercept
X_test['intercept'] = 1.0

In [86]:
# fit models
from sklearn.svm import LinearSVC
svm_classifier = LinearSVC().fit(X_train, A_train)
A_svm_predicted = svm_classifier.predict(X_test)
print A_svm_predicted
svm_classifier = LinearSVC().fit(X_train, B_train)
B_svm_predicted = svm_classifier.predict(X_test)
print B_svm_predicted
svm_classifier = LinearSVC().fit(X_train, C_train)
C_svm_predicted = svm_classifier.predict(X_test)
print C_svm_predicted
svm_classifier = LinearSVC().fit(X_train, D_train)
D_svm_predicted = svm_classifier.predict(X_test)
print D_svm_predicted
svm_classifier = LinearSVC().fit(X_train, E_train)
E_svm_predicted = svm_classifier.predict(X_test)
print E_svm_predicted
svm_classifier = LinearSVC().fit(X_train, F_train)
F_svm_predicted = svm_classifier.predict(X_test)
print F_svm_predicted
svm_classifier = LinearSVC().fit(X_train, G_train)
G_svm_predicted = svm_classifier.predict(X_test)
print G_svm_predicted

[2 1 1 ..., 1 1 0]
[0 0 0 ..., 0 0 0]
[2 4 2 ..., 4 4 4]
[3 3 1 ..., 3 3 3]
[0 0 0 ..., 0 0 0]
[0 0 0 ..., 0 0 0]
[2 3 1 ..., 3 3 3]


In [99]:
#result = pd.concat([A_svm_predicted, B_svm_predicted, C_svm_predicted, D_svm_predicted, E_svm_predicted, F_svm_predicted, G_svm_predicted], axis=1)
#result.to_csv('result5.csv', sep='\t')
#np.savetxt('result5.csv', A_svm_predicted, delimiter=',')
import csv
with open('result5.csv', 'w') as f:
    result_csv = csv.writer(f, delimiter=',')
    for a,b,c,d,e,f,g in zip(A_svm_predicted, B_svm_predicted, C_svm_predicted, D_svm_predicted, E_svm_predicted, F_svm_predicted, G_svm_predicted):
        #print str(a)+str(b)+str(c)+str(d)+str(e)+str(f)+str(g)
        result_csv.writerow(



2023002
1043003
1021001
0012002
1023003
1043003
1013001
1043003
1043002
1013003
1043002
1043003
1043003
2043003
1043003
1043003
0043003
1043003
0033003
1043003
1043023
0043003
1043001
1021001
2043002
1033003
1043002
1043003
1043003
0023002
1023011
1013001
1043023
1043003
1043003
1043001
1043003
0043003
1043003
1043023
1043003
0043001
1043002
1043003
1022022
1043003
1043003
1043003
0043002
1043003
1043003
0043002
0023001
1043023
1043003
1043023
1043003
1043001
1043003
1043003
1043003
0043003
1043003
1043003
1043003
1023003
0023002
1023012
1043003
1043003
1043021
1041002
1013003
1043003
1043003
0033002
1023023
1043003
1043003
1043001
1043003
0043003
1043013
1043022
1043003
1043003
1043002
0033003
1043001
1033023
1033002
1043003
1043002
1043002
1043002
1043003
1023001
1013003
1033003
1043003
1043003
1033002
1043003
1043003
1043003
1043003
1043003
1043003
1043023
0043003
1033023
1043003
1043023
1033002
1043002
1043003
0043003
1043003
1043001
0023002
2043003
1043001
1043003
1041002
1023003
